In [1]:
import cv2
import numpy as np
import time

# Same command function as streaming, its just now we pass in the file path, nice!
cap = cv2.VideoCapture('C:/Users/isabe/Downloads/mice_movie.mp4') # add a 0 to connect to the camera 

# FRAMES PER SECOND FOR VIDEO
fps = 60

# Always a good idea to check if the video was acutally there
# If you get an error at thsi step, triple check your file path!!
if cap.isOpened()== False: 
    print("Error opening the video file. Please double check your file path for typos. Or move the movie file to the same location as this script/notebook")
    
        
# 4 reward areas [(pt1,pt2)]

reward_pts = [[(250, 125), (288, 200)],[(330, 125), (360, 200)],[(250, 310), (288, 380)], [(330, 310), (360, 380)]]

# Create a named window for connections
cv2.namedWindow('Test')

ret, frame = cap.read()
    
mouse= cv2.selectROI(frame, False)
# Initialize tracker with first frame and bounding box
#tracker=cv2.TrackerMedianFlow_create()
ret = tracker(frame, mouse)
  
# While the video is opened
while cap.isOpened():
    
        
    # Read the video file.
    ret, frame = cap.read()
    
    # If we got frames, show them.
    if ret == True:
         
        # Update tracker
        success, roi = tracker.update(frame)
         # roi variable is a tuple of 4 floats
        # We need each value and we need them as integers
        (x,y,w,h) = tuple(map(int,mouse))
    
        ### 
           # Draw Rectangle as Tracker moves
        if success:
        # Tracking success
            p1 = (x, y)
            p2 = (x+w, y+h)
            cv2.rectangle(frame, p1, p2, (0,255,0), 3)
        else :
        # Tracking failure
            cv2.putText(frame, "Failure to Detect Tracking!!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)

         # Display the frame at same frame rate of recording
        time.sleep(1/fps)# only needed for video- not camera
        
        
       ###track mouse###
        
      #  flow = cv2.calcOpticalFlowFarneback(frame1,frame2, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        cv2.imshow('Test',frame)
 
        # Press q to quit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            
            break
            
        frame1=frame2
    # Or automatically break this whole loop if the video is over.
    else:
        break
        
cap.release()
# Closes all the frames
cv2.destroyAllWindows()

numpy.ndarray